# 🤖 Telegram Bot - Google Colab Runner

Run your **Without-id-pw-appx-cp-uploader** Telegram bot directly from Google Colab.

### ⚡ Instructions:
1. Run each cell **one by one** from top to bottom
2. Cell 1 will **auto clone** from GitHub (or **pull updates** if already cloned)
3. Cell 4 starts the bot with **auto-update** — push to GitHub and bot restarts automatically!

> ⚠️ Colab sessions disconnect after ~90 min idle or ~12 hrs max

> 💡 **Workflow:** Edit code locally → `git push` → Bot auto-updates in ~60 seconds!

## 📦 Cell 1: Clone / Update from GitHub + Install Dependencies

In [ ]:
import os

REPO_URL = 'https://github.com/nat-king-15/Without-id-pw-appx-cp-uploader--main.git'
REPO_DIR = '/content/Without-id-pw-appx-cp-uploader--main'

if os.path.exists(REPO_DIR):
    print('📂 Repo already exists — pulling latest updates...')
    !cd {REPO_DIR} && git pull origin main
    print('✅ Updated to latest version!')
else:
    print('📥 Cloning repo from GitHub...')
    !git clone {REPO_URL} {REPO_DIR}
    print('✅ Repo cloned!')

os.chdir(REPO_DIR)
print(f'\n📍 Working directory: {os.getcwd()}')
print(f'📁 Files: {os.listdir(".")}')

## 📦 Cell 2: Install System + Python Dependencies

In [ ]:
# System deps
!apt-get update -qq && apt-get install -y -qq ffmpeg aria2 > /dev/null 2>&1
print('✅ ffmpeg and aria2 installed!')

# Python deps
!pip install -q pyrogram pyromod tgcrypto aiohttp aiofiles cloudscraper \
    pycryptodome requests m3u8 yt-dlp PyPDF2 reportlab motor umongo \
    speedtest-cli flask gunicorn pytz python-dotenv beautifulsoup4 \
    google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client \
    websockets pytube mutagen bs4
print('✅ All Python packages installed!')

## 🔑 Cell 3: Set Environment Variables

Your config values are pre-filled. Change them if needed.

In [ ]:
import os

# ===== BOT CONFIGURATION =====
os.environ['API_ID']    = '6886135'
os.environ['API_HASH']  = 'ee20a1c8a8e44eaa638b7254cbcc3012'
os.environ['BOT_TOKEN'] = '8525803667:AAEjpylk_qaZ62O_gIZmch46S8VQNYxcY1M'
os.environ['PASS_DB']   = '721'

print('✅ Environment variables set!')
print(f'   API_ID:    {os.environ["API_ID"]}')
print(f'   API_HASH:  {os.environ["API_HASH"][:8]}...')
print(f'   BOT_TOKEN: {os.environ["BOT_TOKEN"][:10]}...')

## 🚀 Cell 4: Run Bot with Auto-Update

Bot runs in background and **auto-checks GitHub every 60 seconds**.

When you `git push` new code, the bot will **automatically pull and restart** — no manual work needed!

To stop: click the ⏹️ stop button on this cell.

In [ ]:
import os, subprocess, time, select, fcntl

REPO_DIR = '/content/Without-id-pw-appx-cp-uploader--main'
CHECK_INTERVAL = 60  # seconds between GitHub update checks
os.chdir(REPO_DIR)

# Delete old session file for fresh start
if os.path.exists('bot.session'):
    os.remove('bot.session')
    print('🗑️ Old session file deleted')

def start_bot():
    proc = subprocess.Popen(
        ['python', 'main.py'],
        cwd=REPO_DIR,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        bufsize=1,
        universal_newlines=True
    )
    # Make stdout non-blocking
    fd = proc.stdout.fileno()
    flags = fcntl.fcntl(fd, fcntl.F_GETFL)
    fcntl.fcntl(fd, fcntl.F_SETFL, flags | os.O_NONBLOCK)
    print(f'🚀 Bot started (PID: {proc.pid})')
    return proc

def check_for_updates():
    try:
        result = subprocess.run(
            ['git', 'pull', 'origin', 'main'],
            cwd=REPO_DIR,
            capture_output=True, text=True, timeout=30
        )
        output = result.stdout.strip()
        if 'Already up to date' in output:
            return False
        print(f'\n📦 Update found!')
        print(output)
        return True
    except Exception as e:
        print(f'⚠️ Git check failed: {e}')
        return False

def kill_bot(proc):
    try:
        proc.terminate()
        proc.wait(timeout=5)
    except:
        proc.kill()
    print('⏹️ Bot stopped')

def print_bot_output(proc):
    try:
        while True:
            line = proc.stdout.readline()
            if not line:
                break
            print(line, end='', flush=True)
    except:
        pass

# ===== MAIN LOOP =====
print('=' * 50)
print('🤖 AUTO-UPDATE BOT RUNNER')
print(f'⏱️  Checking GitHub every {CHECK_INTERVAL}s')
print(f'📂 {REPO_DIR}')
print('=' * 50)

bot_proc = start_bot()

try:
    while True:
        # Print bot output for CHECK_INTERVAL seconds
        start_time = time.time()
        while time.time() - start_time < CHECK_INTERVAL:
            if bot_proc.poll() is not None:
                print('\n💀 Bot crashed! Restarting in 5s...')
                time.sleep(5)
                bot_proc = start_bot()
                break
            print_bot_output(bot_proc)
            time.sleep(1)

        # Check for GitHub updates
        if check_for_updates():
            print('\n🔄 New update detected! Restarting bot...')
            kill_bot(bot_proc)
            time.sleep(2)
            bot_proc = start_bot()
            print('✅ Bot restarted with latest code!\n')

except KeyboardInterrupt:
    print('\n🛑 Stopping...')
    kill_bot(bot_proc)
    print('👋 Done!')

---
## 🔄 Optional: Keep Colab Alive

Open browser console (F12 → Console) and paste:
```javascript
function KeepAlive() {
    console.log('Keeping alive...');
    document.querySelector('colab-connect-button').click();
}
setInterval(KeepAlive, 60000);
```